# Lab - 7 
## SHA_512 Client

In [8]:
from Crypto.Cipher import AES 
from Crypto.Util.Padding import pad, unpad
import socket
from hashlib import sha512

def format_key(key):
    return key.ljust(16)[:16].encode()

def encrypt(message, key):
    key = format_key(key)  
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return cipher.encrypt(pad(message.encode(), AES.block_size))

def decrypt(ciphertext, key):
    key = format_key(key)
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ciphertext), AES.block_size).decode()

def hash512(data):
    output = sha512(data.encode()).hexdigest()
    return (output)

In [9]:
import socket
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

def client():
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('localhost', 1234))

    client_private_key = ec.generate_private_key(ec.SECP256R1())
    client_public_key = client_private_key.public_key()

    server_public_bytes = client.recv(1024)
    server_public_key = serialization.load_pem_public_key(server_public_bytes)

    client_public_bytes = client_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    client.sendall(client_public_bytes)
    
    shared_secret = client_private_key.exchange(ec.ECDH(), server_public_key)
    print("Client derived shared secret:", shared_secret.hex())
    
    decrypt_text = decrypt(client.recv(1024),shared_secret.hex())
    msg,hashcode=decrypt_text.split('***')
    hash2=hash512(msg)
    
    if(hash2 == hashcode):
        print('Integrity is maintained!')
        print("Your message: ",msg)
    else:
        print('No Integrity')
    client.close()

client()

Client derived shared secret: 3c859a3f900095db5248b45c8410ca1bdafbb2144b486f51ccbab4079c465d03
Integrity is maintained!
Your message:  hello client
